# COPY INTO and MERGE commands
* Incrementally loads data into Delta Lake tables from Cloud storage
* Supports schema evolution
* Supports wide range of file formats
* Alternative to Auto Loader for batch ingestion

Documentation: [COPY INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into)

## Create table to copy data into

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

## Incrementally load new files into the table

In [0]:
%sql
DELETE FROM demo.delta_lake.raw_stock_prices;

COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@maideacourseextdl.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql 
SELECT * FROM demo.delta_lake.raw_stock_prices;

## MERGE INTO statement
* Used for upserts
* Allows mergin new data into a target table based on matching condition

Documentation: [MERGE INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-merge-into)

### Create the table to merge data into

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

### Merge the source data into target table
1. Insert new stocks received
2. Update price and trading_date if updates received
3. Delete stocks which are delisted from the exchange (status=DELISTED)

In [0]:
%sql
MERGE INTO demo.delta_lake.stock_prices AS target
USING demo.delta_lake.raw_stock_prices AS source
  ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN -- update existing stocks
  UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN -- delete delisted stocks
  DELETE
WHEN NOT MATCHED AND source.status = 'ACTIVE' THEN -- insert new stocks
  INSERT (stock_id, price, trading_date) 
  VALUES (source.stock_id, source.price, source.trading_date);

In [0]:
%sql
select * from demo.delta_lake.stock_prices